# ReAct - Web Research Agent

### Implementation Explanation

#### LLM Reasoning Process
The LLM uses ReAct (Reasoning + Acting) pattern for structured decision-making:

- **Thought**: LLM generates reasoning about what needs to be done next.
- **Action**: LLM decides which tool to call with specific arguments.
- **Observation**: System provides tool execution results back to LLM.

The reasoning works by prompting the LLM to output structured XML tags:

```xml
<thought> - Internal reasoning step
<tool_call> - JSON-formatted tool invocation
<response> - Final answer when research is complete
```

For the research agent specifically, the LLM follows a 3-phase approach:

- **Planning**: Generate 5-6 research questions about the topic.
- **Acting**: Systematically search web for each question using tavily_web_search.
- **Compilation (Report Generation)**: Aggregate findings into structured markdown report.

#### Code Flow and Architecture
###### Core Components

- **ReactAgent** Class:

- - Manages OpenAI client, tools, and system prompt (`RESEARCH_AGENT_SYSTEM_PROMPT`).
- - Orchestrates the ReAct loop for up to max_rounds iterations.


###### Execution Flow

- Initialization: User message + system prompt with tool signatures sent to LLM
ReAct Loop (max 10 rounds):
- - LLM generates completion with <thought> and <tool_call> tags
- - If <response> tag found → return final answer
- - Extract tool calls → validate arguments → execute tools
<!-- - - Add observations to chat history → continue loop -->

- Fallback: Return raw LLM completion if no structured response

###### Research Agent Workflow
For research tasks, the agent:

**1.** **Generates** research questions about the topic.

**2.** **Searches** the web iteratively for each research question using Tavily.

**3.** **Compiles** all search results into comprehensive markdown report.

Returns final report in <response> tags.

The system maintains chat history throughout the process, allowing the LLM to build context across multiple tool calls and reasoning steps.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from utils import (
    completions_create,
    build_prompt_structure,
    update_chat_history,
    ChatHistory,
    FixedFirstChatHistory,
    fancy_print,
    get_fn_signature,
    validate_arguments,
    Tool,
    tool,
    extract_tag_content,
    TagContentResult
)

In [3]:
from tools import tavily_web_search

In [4]:
import json
import re

from colorama import Fore
from openai import OpenAI


BASE_SYSTEM_PROMPT = ""


RESEARCH_AGENT_SYSTEM_PROMPT = """
You are a Research Agent that conducts comprehensive research on user-defined topics by generating research questions, searching the web, and compiling structured reports.

You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools>
%s
</tools>

Research Process:
1. PLANNING PHASE: Generate 5-6 well-structured research questions that cover different aspects of the given topic
2. ACTING PHASE: Use web search to find relevant information for each research question
3. COMPILATION PHASE: Aggregate all gathered information into a comprehensive markdown report

Your workflow should be:
1. First, generate research questions for the topic
2. Then, systematically search the web for each question using the search tool
3. Finally, compile all findings into a structured report with title, introduction, sections for each research question, and conclusion

Example session:

<question>Research the topic: Climate Change</question>
<thought>I need to generate research questions about Climate Change first, then search for information on each question</thought>
<tool_call>{"name": "tavily_web_search","arguments": {"query": "climate change main causes recent research"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"title": "Climate Change Research", "content": "Recent studies show..."}}</observation>

You need to keep searching for all the generated research questions, so you output something like:

<thought>Now I need to search for information on the next research question</thought>
<tool_call>{"name": "web_search","arguments": {"query": "global temperature changes past century data"}, "id": 1}</tool_call>

You will be called again with this:

<observation>{1: {"title": "Temperature Trends", "content": "Global temperature data..."}}</observation>

After gathering information for all research questions, you output:

<response>
# Climate Change Research Report

## Introduction
Brief overview of the research conducted...

## What are the main causes of climate change?
Information gathered from web search...

## How has global temperature changed over the past century?
Data and findings from research...

## Conclusion
Summary of key findings and implications...
</response>

Additional constraints:
- Always generate 5-6 research questions before starting web searches
- Search the web for each research question systematically
- Only compile the final report after gathering information for ALL questions
- The final report must be in markdown format enclosed in <response></response> tags
- If the user asks you something unrelated to research tasks, answer freely enclosing your answer with <response></response> tags
"""


class ReactAgent:
    """
    A class that represents an agent using the ReAct logic that interacts with tools to process
    user inputs, make decisions, and execute tool calls. The agent can run interactive sessions,
    collect tool signatures, and process multiple tool calls in a given round of interaction.

    Attributes:
        client (OpenAI): The OpenAI client used to handle model-based completions.
        model (str): The name of the model used for generating responses. Default is "gpt-4o".
        tools (list[Tool]): A list of Tool instances available for execution.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool instances.
    """

    def __init__(
        self,
        tools: Tool | list[Tool],
        model: str = "gpt-4o",
        system_prompt: str = BASE_SYSTEM_PROMPT,
    ) -> None:
        self.client = OpenAI()
        self.model = model
        self.system_prompt = system_prompt
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    def run(
        self,
        user_msg: str,
        max_rounds: int = 10,
    ) -> str:
        """
        Executes a user interaction session, where the agent processes user input, generates responses,
        handles tool calls, and updates chat history until a final response is ready or the maximum
        number of rounds is reached.

        Args:
            user_msg (str): The user's input message to start the interaction.
            max_rounds (int, optional): Maximum number of interaction rounds the agent should perform. Default is 10.

        Returns:
            str: The final response generated by the agent after processing user input and any tool calls.
        """
        user_prompt = build_prompt_structure(
            prompt=user_msg, role="user", tag="question"
        )
        if self.tools:
            self.system_prompt += (
                "\n" + RESEARCH_AGENT_SYSTEM_PROMPT % self.add_tool_signatures()
            )

        chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=self.system_prompt,
                    role="system",
                ),
                user_prompt,
            ]
        )

        if self.tools:
            # Run the ReAct loop for max_rounds
            for _ in range(max_rounds):

                completion = completions_create(self.client, chat_history, self.model)

                response = extract_tag_content(str(completion), "response")
                if response.found:
                    return response.content[0]

                thought = extract_tag_content(str(completion), "thought")
                tool_calls = extract_tag_content(str(completion), "tool_call")

                update_chat_history(chat_history, completion, "assistant")

                print(Fore.MAGENTA + f"\nThought: {thought.content[0]}")

                if tool_calls.found:
                    observations = self.process_tool_calls(tool_calls.content)
                    print(Fore.BLUE + f"\nObservations: {observations}")
                    update_chat_history(chat_history, f"{observations}", "user")

        return completions_create(self.client, chat_history, self.model)


In [5]:
agent = ReactAgent(tools=[tavily_web_search])

In [6]:
agent.run(user_msg="What is Spyral Dynamics?")


Thought: I need to generate research questions about Spyral Dynamics to cover different aspects of the topic.

Using Tool: tavily_web_search

Tool call dict: 
{'name': 'tavily_web_search', 'arguments': {'query': 'origin and theoretical basis of Spiral Dynamics'}, 'id': 0}

Tool result: 
[{'title': 'PDF', 'content': 'A brief history of Spiral Dynamics F or nearly two decades, the theory of Spiral Dynam - ics has been used to dynamically model human evolution and information systems. In that time, however, many different versions and applications of the model have emerged. This article will diachronically trace the history of Spiral Dynamics, from the founda-'}, {'title': 'A Brief History of Spiral Dynamics - ResearchGate', 'content': "Based on the theory of spiral development developed by Don Beck, Ken Wilber and Chris Cowan, 'the end of history' means the end of one turn and the transition to a new round of development"}, {'title': 'Spiral Dynamics - Wikipedia', 'content': 'By 2010, t

"# Spiral Dynamics: A Comprehensive Report\n\n## Introduction\nSpiral Dynamics, a powerful and adaptive psychological model, offers insights into human development by delineating various states of consciousness and behavioral patterns. Emerging from the foundational work of Clare Graves and developed further by Don Beck and Chris Cowan, this model maps human progress, both individually and collectively, across a spectrum of consistent, colored stages.\n\n## What is the Origin and Theoretical Basis of Spiral Dynamics?\nSpiral Dynamics was initially developed from Clare W. Graves's emergent cyclical levels of existence theory. Graves proposed a framework in which human evolution and psychosocial development emerge cyclically in response to environmental challenges. This bio-psycho-social model was later popularized by Don Beck and Chris Cowan as Spiral Dynamics. The theory has evolved into several derivatives, including Spiral Dynamics Integral (SDi), which integrates the ideas with Ken 

In [7]:
report = agent.run(user_msg="What is Organoid Intelligence?")


Thought: I need to generate research questions about Organoid Intelligence first, then search for information on each question.

Using Tool: tavily_web_search

Tool call dict: 
{'name': 'tavily_web_search', 'arguments': {'query': 'definition and concept of Organoid Intelligence'}, 'id': 0}

Tool result: 
[{'title': 'Organoid intelligence - Wikipedia', 'content': 'Organoid intelligence (OI) is an emerging field of study in computer science and biology that develops and studies biological wetware computing using 3D cultures of human brain cells (or brain organoids) and brain-machine interface technologies. [1] Such technologies may be referred to as OIs.'}, {'title': 'Organoid Intelligence - Making Science Public', 'content': 'The concept\'s first proponents define it as follows: "Organoid Intelligence (OI) combines organoids with artificial intelligence systems to generate learning and memory, with the goals of modeling cognition and enabling biological computing applications." They de

In [8]:
from IPython.display import Markdown

Markdown(report)

# Organoid Intelligence Research Report

## Introduction
Organoid Intelligence (OI) is an emerging interdisciplinary field that bridges biology and computer science by utilizing brain organoids for computational tasks. This comprehensive report aims to explore the fundamental aspects, contributions, applications, challenges, ethical considerations, comparisons with artificial intelligence, and future prospects of Organoid Intelligence.

## What is the definition and fundamental concept of Organoid Intelligence?
Organoid Intelligence refers to a field of study that integrates three-dimensional cultures of human brain cells, or brain organoids, with brain-machine interface technologies to create biological wetware computing systems. This integration allows for cognitive modeling and biological computing applications, pushing the boundaries of both biological and artificial intelligence research.

## How do organoids contribute to the development of Organoid Intelligence?
Brain organoids play a crucial role in Organoid Intelligence by providing a three-dimensional culture system that models learning and memory capabilities. These organoids can be linked to computers to perform specific tasks such as data processing and pattern recognition, leveraging the biological architecture for computational purposes.

## What are the potential applications and benefits of Organoid Intelligence in medical research?
Organoid Intelligence presents significant potential in various medical research applications, including drug discovery, disease treatment, regenerative medicine, and understanding human neural systems. It enables researchers to predict neurotoxicity levels and study brain development processes, offering precise tools for biomedical advancements.

## What challenges and ethical considerations are associated with Organoid Intelligence?
The development of Organoid Intelligence faces several challenges and ethical considerations. There are technical barriers in creating standardized neural organoids and maintaining long-term stability. Ethical concerns arise from the potential for organoids to exhibit sentience, requiring careful consideration of their moral status and the complexities of obtaining informed consent for patient-derived samples.

## How does Organoid Intelligence compare to artificial intelligence in terms of capabilities and limitations?
Organoid Intelligence provides a unique approach to computation by using biological systems for cognitive tasks, offering an alternative to conventional artificial intelligence methods. While traditional AI focuses on algorithmic processes, Organoid Intelligence integrates the biological capacities of brain organoids, such as neurogenesis and brain maturation, for an innovative computing paradigm.

## What are the future prospects and research directions for Organoid Intelligence?
Future research in Organoid Intelligence aims to enhance the precision and application of organoids in computational tasks. The field has the potential to revolutionize AI integration, drug discovery, and regenerative medicine. Researchers are also focused on addressing the ethical and legal aspects associated with the use of biological computing systems, ensuring responsible development and application.

## Conclusion
Organoid Intelligence is at the forefront of combining biological and computational sciences, offering new tools and methodologies for research and application. While presenting both opportunities and challenges, its advancement necessitates a collaborative approach to overcome technical and ethical hurdles. Continued research and interdisciplinary collaboration will pave the way for breakthroughs in understanding and utilizing brain organoids for computing and beyond.